In [1]:
import pandas
import datetime
import os
import numpy
from scipy import signal, ndimage
from davitpy.models import *
from davitpy import utils
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
import seaborn as sns
%matplotlib inline

In [ ]:
selDT = datetime.datetime( 2011, 4, 9, 8, 40 )
medFiltedSize = ( 1, 3 )
fileDtStr = selDT.strftime("%y%m%d")
# Load the appropriate file based on date
baseDir = "../data/"
for root, dirs, files in os.walk(baseDir):
    for fName in files:
        if fName.find(fileDtStr) != -1:
            inpTecFile = root + "/" + fName

In [ ]:
dataDF = pandas.read_hdf(inpTecFile, 'Data/Table Layout')
dataDF["date"] = pandas.to_datetime(dataDF["year"]*10000000000 +\
                                    dataDF["month"]*100000000 + dataDF["day"]*1000000 +\
                                    dataDF["hour"]*10000 + dataDF["min"]*100 +\
                                    dataDF["sec"],format='%Y%m%d%H%M%S')
# Only take selected datetime
dataDF["selTimeDel"] = dataDF["date"] - selDT
dataDF = dataDF[ dataDF["selTimeDel"] == min(abs(dataDF["selTimeDel"])) ].reset_index(drop=True)
# Discard tec values which are 2stds away!
cutOffLimit = dataDF["tec"].mean() + 2*dataDF["tec"].std()
dataDF = dataDF[ dataDF["tec"] <= cutOffLimit ]
dataDF.head()

In [ ]:
medDF = dataDF.pivot( index="gdlat", columns="glon", values="tec" )
filteredTecArr = signal.medfilt( medDF.as_matrix(), kernel_size=medFiltedSize )
medDF[ medDF.columns ] = filteredTecArr
medDF = medDF.unstack().reset_index()
medDF.columns = [ "glon", "gdlat", "med_tec" ]

In [ ]:
gLonArr = medDF["glon"].values
gdLatArr = medDF["gdlat"].values
mlon, mlat = utils.coord_conv( gLonArr, gdLatArr, \
                                 "geo", "mag", altitude=300., \
                                 date_time=selDT )
medDF["mlon"] = mlon
medDF["mlat"] = mlat
medDF["mlt"] = [ aacgm.mltFromYmdhms(selDT.year, \
                selDT.month,selDT.day, selDT.hour,\
                selDT.minute, selDT.second, x) for x in medDF["mlon"] ]
medDF["normMLT"] = [x-24 if x >= 12\
             else x for x in medDF['mlt']]
medDF.head()

In [ ]:
# set colorbar
seaMap = ListedColormap(sns.color_palette("RdBu_r"))
# Plot map
f = plt.figure(figsize=(12, 8))
ax = f.add_subplot(1,1,1)
m1 = utils.plotUtils.mapObj(boundinglat=30., gridLabels=True, coords='mag', ax=ax, datetime=selDT)
xVec, yVec = m1(list(medDF["mlon"]), list(medDF["mlat"]), coords="mag")
tecPlot = m1.scatter( xVec, yVec , c=medDF["med_tec"], s=40.,\
           cmap=seaMap, alpha=0.7, zorder=5., \
                     edgecolor='none', marker="s" )

cbar = plt.colorbar(tecPlot, orientation='vertical')
cbar.set_label('TEC', size=15)
f.savefig("../figs/tec-med-filt-" + selDT.strftime("%Y%m%d-%H%M") + ".pdf",bbox_inches='tight')

In [ ]:
# medDF = medDF.round(2)
test = medDF.groupby( "mlt" ).count()
print medDF.head()